In [ ]:
using Plots

In [ ]:
struct FDMesh2D
    xl::Float64
    xr::Float64
    yl::Float64
    yr::Float64
    dx::Float64
    dy::Float64
    nx::Int64
    ny::Int64
    # nodes::Vector{Tuple{Float64, Float64}}
    nodes_idxs::CartesianIndices{2}
    midpoints_idxs_x::CartesianIndices{2} # On horizontal edges, staggered on x axis
    midpoints_idxs_y::CartesianIndices{2} # On vertical edges, staggered on y axis
end

In [ ]:
function FDMesh2D(xl::Float64, xr::Float64, nx::Int64, yl::Float64, yr::Float64, ny::Int64)
    hx = (xr - xl) / nx
    hy = (yr - yl) / ny
    metrics = [hx, hy]
    nodes_idxs = CartesianIndices((nx + 1, ny + 1))
    midpoints_idxs_x = CartesianIndices((nx, ny + 1))
    midpoints_idxs_y = CartesianIndices((nx + 1, ny))
    return FDMesh2D(xl, xr, yl, yr, hx, hy, nx, ny, nodes_idxs, midpoints_idxs_x, midpoints_idxs_y)
    # nodes = [(collect(Tuple(idx)).-1) .* metrics for idx in nodes_idxs] |> vec
end  

In [ ]:
xl, xr = 0.0, 1.0
yl, yr = 0.0, 1.0
nx, ny = 3, 3
mesh = FDMesh2D(xl ,xr, nx, yl, yr, ny)
println(mesh.nodes_idxs)
println(mesh.midpoints_idxs_x)
println(mesh.midpoints_idxs_y)

In [ ]:
function gridPointToLeftEdgeNeighbor(mesh::FDMesh2D, x::Int, y::Int)
    mesh.nx * (y - 1) + (x - 1)
end

function gridPointToRightEdgeNeighbor(mesh::FDMesh2D, x::Int, y::Int)
    mesh.nx * (y - 1) + x
end

function gridPointToTopEdgeNeighbor(mesh::FDMesh2D, x::Int, y::Int)
    (mesh.nx + 1) * (y - 1) + x 
end

function gridPointToBottomEdgeNeighbor(mesh::FDMesh2D, x::Int, y::Int)
    (mesh.nx + 1) * (y - 2) + x 
end

In [ ]:
function generateEtaMatrix(mesh::FDMesh2D)
    eta_start = 0
    u_start = eta_start + (mesh.nx + 1) * (mesh.ny + 1)
    v_start = u_start + (mesh.nx) * (mesh.ny + 1)
    A = zeros(Float64, (mesh.nx + 1) * (mesh.ny + 1), (mesh.nx + 1) * (mesh.ny + 1) + (mesh.nx) * (mesh.ny + 1) + (mesh.nx + 1) * (mesh.ny))

    
    i = 0
    for idx in mesh.nodes_idxs
        i += 1    
        x, y = Tuple(idx)      
        if x == 1
            # Left boundary, including corners!
            continue
        elseif y == 1 && x == (mesh.nx + 1)
            # Bottom right corner
            continue
        elseif y == (mesh.ny + 1) && x == (mesh.nx + 1)
            # Top right corner
            continue
        elseif y == 1
            # Bottom boundary
            continue
        elseif y == (mesh.ny + 1)
            # Top boundary
            continue
        elseif x == (mesh.nx + 1)
            # Right boundary
            continue
        else
            # Inside
            left_midpoint = gridPointToLeftEdgeNeighbor(mesh, x, y)
            right_midpoint = gridPointToRightEdgeNeighbor(mesh, x, y)
            top_midpoint = gridPointToTopEdgeNeighbor(mesh, x, y)
            bottom_midpoint = gridPointToBottomEdgeNeighbor(mesh, x, y)
            A[i,u_start + right_midpoint] = 1/mesh.dx 
            A[i,u_start + left_midpoint] = -1/mesh.dx 
            A[i,v_start + top_midpoint] = 1/mesh.dy
            A[i,v_start + bottom_midpoint] = -1/mesh.dy
        end
            
    end
    A
end

In [ ]:
A = generateEtaMatrix(mesh)
Base.print_matrix(stdout, A)